In [ ]:
!pip install pinecone-client==3.2.2 sentence-transformers==2.7.0 datasets==2.19.0 transformers==4.40.1 openai==1.25.2 llama-index==0.10.34 llama-index-vector-stores-pinecone==0.1.6  -qqq
# faiss-cpu==1.7.2
!pip install -q condacolab
import condacolab
condacolab.install()

!conda install -c pytorch faiss-cpu -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 141.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.

# **실습 데이터 다운로드**

In [33]:
!wget ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
!tar -xf sift.tar.gz
!mkdir data/sift1M -p
!mv sift/* data/sift1M

--2025-03-18 08:32:22--  ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
           => ‘sift.tar.gz.1’
Resolving ftp.irisa.fr (ftp.irisa.fr)... 131.254.254.45, 2001:660:7303:254::45
Connecting to ftp.irisa.fr (ftp.irisa.fr)|131.254.254.45|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /local/texmex/corpus ... done.
==> SIZE sift.tar.gz ... 168280445
==> PASV ... done.    ==> RETR sift.tar.gz ... done.
Length: 168280445 (160M) (unauthoritative)

sift.tar.gz.1       100%[===================>] 160.48M  20.4MB/s    in 9.6s    

2025-03-18 08:32:34 (16.7 MB/s) - ‘sift.tar.gz.1’ saved [168280445]



# **실습 데이터 불러오기**

In [34]:
import psutil

def get_memory_usage_mb():
  # 현재 실행 중인 프로세스 정보 가져오기
  process = psutil.Process()
  # 프로세스의 메모리 사용량 정보 가져오기
  memory_info = process.memory_info()

  # memory_info.rss -> Resident Set Size(RSS) -> 프로세스가 실제로 물리적 메모리에 로드된 크기
  # 바이트를 MB로 변환
  return memory_info.rss / (1024 * 1024)

In [35]:
import time
import faiss
from faiss.contrib.datasets import DatasetSIFT1M

# DatasetSIFT1M -> FAISS에서 제공하는 SIFT1M 데이터셋을 로드하는 클래스
# SIFT1M -> 100만 개(1M) SIFT(Scale-Invariant Feature Transform) 벡터로 구성된 데이터셋
# 이미지 검색, 유사도 검색 등의 벤치마킹에 많이 사용됨
ds = DatasetSIFT1M()

# 검색에 사용할 데이터
xq = ds.get_queries()
# 저장된 벡터 데이터
xb = ds.get_database()
# 질문에 대한 실제 정답 데이터
gt = ds.get_groundtruth()

# **데이터가 늘어날 때 색인/검색 시간, 메모리 사용량 변화**

In [28]:
k = 1
# 벡터 차원 추출 (128)
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

# i를 2씩 증가
for i in range(1, 10, 2):
  start_memory = get_memory_usage_mb()
  start_indexing = time.time()

  index = faiss.IndexFlatL2(d)
  index.add(xb[:(i + 1) * 100000])

  end_indexing = time.time()
  end_memory = get_memory_usage_mb()

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  print(f"데이터 {(i + 1) * 100000}개:")
  print(f"색인: {(end_indexing - start_indexing) * 1000 :.3f} ms ({end_memory - start_memory:.3f} MB) 검색: {(t1 - t0) * 1000 / nq :.3f} ms")

ValueError: input not a numpy array

# **파라미터 m의 변경에 따른 성능 확인**

In [38]:
import numpy as np

k = 1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

# 강제 변환 (안전하게)
xb = np.ascontiguousarray(xb, dtype=np.float32)
print(f"xb type: {type(xb)}")
print(f"xb dtype: {xb.dtype}")
print(f"xb shape: {xb.shape}")
print(f"Is xb C-contiguous? {xb.flags['C_CONTIGUOUS']}")

# xq = np.ascontiguousarray(xq, dtype=np.float32)
# print(f"xq type: {type(xq)}")
# print(f"xq dtype: {xq.dtype}")
# print(f"xq shape: {xq.shape}")
# print(f"Is xq C-contiguous? {xq.flags['C_CONTIGUOUS']}")

for m in [8, 16, 32, 64]:
  index = faiss.IndexHNSWFlat(d, m)

  time.sleep(3)

  start_memory = get_memory_usage_mb()
  start_index = time.time()

  try:
    index.add(xb)
    print("FAISS IndexHNSWFlat 생성 및 xb 추가 성공!")
  except ValueError as e:
    print(f"IndexHNSWFlat 인덱스 추가 중 에러 발생: {e}")

  end_memory = get_memory_usage_mb()
  end_index = time.time()

  print(f"M: {m} - 색인 시간: {end_index - start_index} s, 메모리 사용량: {end_memory - start_memory} MB")

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  # gt[:nq, :1] -> gt[:1000, :1] -> 즉 추린 질의 1000개 만큼의 정답 데이터 1000개를 가져와 실제 정답 인덱스인 1을 추출
  # 정답 데이터를 모두 더해서 전체 쿼리 개수로 나눠 계산한다
  recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)
  print(f"{(t1 - t0) * 1000.0 / nq:.3f} ms per query, R@1 {recall_at_1:.3f}")

xb type: <class 'numpy.ndarray'>
xb dtype: float32
xb shape: (1000000, 128)
Is xb C-contiguous? True
IndexHNSWFlat 인덱스 추가 중 에러 발생: input not a numpy array
M: 8 - 색인 시간: 0.0004429817199707031 s, 메모리 사용량: 0.0 MB


ValueError: input not a numpy array